In [14]:
import time
import datetime
import pandas as pd 
import pyodbc
import sys
import sqlalchemy
import nbimporter
pd.set_option('display.max_columns', 25)
driver = 'ODBC Driver 17 for SQL Server'


def execute_sql(sql):

    cursor.execute(sql)
    try:
        column_list = [column[0] for column in cursor.description]
        row_list = [row for row in cursor]
        table = pd.DataFrame.from_records(row_list, columns=column_list)
        print ("Query executed.")
        return table
    except:
        print ("Query executed.")
        return
    
def push_df_sql(server, db, table_name, dataframe,option='replace'):

    """ Sends pandas dataframe to specified table in SQL Server
        sqlalchemy required b/c pyodbc in execute_sql() does not support "to sql" cmds """

    # connect to SQL Server via sqlalchemy by specifying driver/server/database
    conn_string = 'mssql+pyodbc://' + server + '/' + db + \
                    '?trusted_connection=yes&driver=' + driver

    # initialize engine to establish connection to server
    engine = sqlalchemy.create_engine(conn_string, isolation_level='READ_UNCOMMITTED')
    connection = engine.connect()

    print('Pushing df into {}.dbo.{}'.format(db, table_name))

    # push pandas df to table in sql server
    dataframe.to_sql(table_name, con=engine, if_exists=option, 
        index=False, schema='dbo', chunksize=10)

    # close connection
    connection.close()

    print('--------------------')
    print('Pushed successfully')
    print('\n')
def push_junk(server,df,table_name,option='replace'):
    database = 'csn_junk' ####Enter database here
    
    print('Connecting to '+server+'...')
    print('---------------------')
    con_string = 'mssql+pyodbc://{0}/{1}?trusted_connection=yes&driver=ODBC+Driver+13+for+SQL+Server&MultiSubnetFailover=yes'.format(server,database)     
    engine = sqlalchemy.create_engine(con_string, echo=False,isolation_level='READ_UNCOMMITTED')
    con = engine.connect()
    
    print('Pushing junk table '+table_name+'...')
    print('---------------------')
    df.to_sql(name=table_name, con=engine, schema = 'dbo', if_exists=option,index=False)
    
    print('Table pushed!')
    con.close()
    engine.dispose()

def connect_db(server,db):
    import sqlalchemy
    print('Connecting to '+server+'...')
    conn = pyodbc.connect("DRIVER={" + driver + "};SERVER=" + server + "; DATABASE=" + db + "; Trusted_Connection=yes;MultiSubnetFailover=Yes",autocommit=True)
    cursor = conn.cursor()
    print ('Connected to '+server+'...')
    return conn,cursor

# Read SQL scripts in WPP folder
def read_sql(query_name):
    with open('G:\\Shared drives\\Competitive Intelligence\\8. Exclusivity\\3. Ongoing Projects\\WPP Products\\SQL\\PIE Dashboard\\{}'.format(query_name)) as query_file:
        return query_file.read()
    
import datetime
d = datetime.date.today()
day = d.strftime("%d")
month = d.strftime("%m")
year = d.strftime("%y")

#### Verifying month & year for tables and files

In [2]:
print("Current month set to: ",month)
a =  input('''Verify the month of the report by entering the number\n
NOTE: All existing reports/tables with same month + year will be overwritten\n''')
print(a)
if len(a) == 1:
    a = '0' + a
if a != month:
    month = a

Current month set to:  12
12


In [3]:
print("Current year set to: ",year)
a =  input('''Verify the year of the report by entering the number in YY format\n
NOTE: All existing reports/tables with same month + year will be overwritten\n''')
print(a)
if len(a) == 1:
    a = '0' + a
if len(a) == 4:
    a = a[2:]
if a != year:
    year = a

Current year set to:  19
19


In [13]:
print("Current month set to: ",month)
print("Current year set to: ",year)
#Define tables for reports
ed_buckets_name = 'ED_buckets_{0}{1}'.format(month,year)
print("Violation buckets table name:",pie_buckets_name)

Current month set to:  12
Current year set to:  19
Violation buckets table name: PIE_buckets_1219


In [5]:
#Connect to SQLMERCHANDISING
conn,cursor = connect_db('SQLMERCHANDISING','csn_junk')

Connecting to SQLMERCHANDISING...
Connected to SQLMERCHANDISING...


In [8]:
ED_buckets_query = read_sql('Supplier buckets v4.sql').replace('mm',month).replace('yy',year) 
ED_buckets_df = execute_sql(ED_buckets_query)

Query executed.


In [9]:
ED_buckets_df.groupby('ViolationStatus').nunique()

,SupplierPartID,SupplierPartNumber,SupplierID,PrSKU,Supplier,ViolationStatus,WayfairURL,AmazonURL,AgreementType,WPP_Supplier,Supplier_Exclusive_GRS
ViolationStatus,,,,,,,,,,,
3rd Party Seller,29144,29099,510,16539,510,1,16539,30814,3,2,510
Amazon as Seller,18493,18462,346,7866,346,1,7866,18613,3,2,346
No Detection,51284,51235,540,19795,540,1,19795,0,3,2,540


In [11]:
ED_buckets_df.groupby(['AgreementType','ViolationStatus']).nunique()

SupplierPartID  SupplierPartNumber  \
AgreementType   ViolationStatus                                        
FlagshipBrands  3rd Party Seller           10462               10462   
                Amazon as Seller            6121                6121   
                No Detection               21304               21272   
LifestyleBrands 3rd Party Seller             210                 210   
                Amazon as Seller             123                 123   
                No Detection                  42                  42   
WPP             3rd Party Seller           18511               18473   
                Amazon as Seller           12292               12262   
                No Detection               29948               29933   

                                  SupplierID  PrSKU  Supplier  \
AgreementType   ViolationStatus                                 
FlagshipBrands  3rd Party Seller         358   4852       358   
                Amazon as Seller         237   2334       237   
                No Detection             341   3433       341   
LifestyleBrands 3rd Party Seller          20    107        20   
                Amazon as Seller          16     68        16   
                No Detection              14     26        14   
WPP             3rd Party Seller         358  11601       358   
                Amazon as Seller         240   5484       240   
                No Detection             403  16343       403   

                                  ViolationStatus  WayfairURL  AmazonURL  \
AgreementType   ViolationStatus                                            
FlagshipBrands  3rd Party Seller                1        4852      11579   
                Amazon as Seller                1        2334       6166   
                No Detection                    1        3433          0   
LifestyleBrands 3rd Party Seller                1         107        246   
                Amazon as Seller                1          68        123   
                No Detection                    1          26          0   
WPP             3rd Party Seller                1       11601      19053   
                Amazon as Seller                1        5484      12375   
                No Detection                    1       16343          0   

                                  AgreementType  WPP_Supplier  \
AgreementType   ViolationStatus                                 
FlagshipBrands  3rd Party Seller              1             2   
                Amazon as Seller              1             2   
                No Detection                  1             2   
LifestyleBrands 3rd Party Seller              1             2   
                Amazon as Seller              1             2   
                No Detection                  1             2   
WPP             3rd Party Seller              1             2   
                Amazon as Seller              1             2   
                No Detection                  1             2   

                                  Supplier_Exclusive_GRS  
AgreementType   ViolationStatus                           
FlagshipBrands  3rd Party Seller                     358  
                Amazon as Seller                     237  
                No Detection                         341  
LifestyleBrands 3rd Party Seller                      20  
                Amazon as Seller                      16  
                No Detection                          14  
WPP             3rd Party Seller                     358  
                Amazon as Seller                     240  
                No Detection                         403

In [15]:
push_junk('SQLMERCHANDISING',ED_buckets_df,ed_buckets_name)

Connecting to SQLMERCHANDISING...
---------------------
Pushing junk table ED_buckets_1219...
---------------------
Table pushed!
